<center><img src='https://drive.google.com/uc?export=view&id=1qJ8NqAZolTBQY7lN-deZ8xEsU3dlUiLz' width=200></center>


<h6><center></center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Recherche d'Information et traitement de données massives </center>
    <center> Lab 7 : Personnalisation : Systèmes de recommandation </center>
<hr style=" border:none; height:3px;">
</h1>

Ce LAB a pour objectif de vous familiariser avec le problème de la recommandation par la mise en place de la chaîne de traitements pour la réalisation d'un système de recommandation de films.  

## Avant de démarrer

Dans ce lab nous allons utiliser la bibliothèque Pandas. Pour la découvir (si ce n'est pas déjà fait), nous vous proposons un petit tuto que vous pouvez récupérer [ici](https://github.com/wassilaOuerdane/Lab7_TutoPanda_2021.git)

## Démarrage du Lab7

Comme pour les précédents labs, pour le cas du lab effectué sur colab, voici les **instructions pour permettre l'importation du dossier Data fournie avec ce lab :**

1.   Dans le dossier Lab7_Recom-2021 téléchargé depuis git, vous trouverez une archive Archive_Lab7.zip.
2.   Ouvrir le panneau Fichiers de colab (c'est à dire cliquez sur le logo en forme de dossier à gauche) et cliquez sur le premier bouton en haut à gauche vous permettant de télécharger. Téléchargez ainsi l'Archive_Lab7.zip dans votre colab. Si besoin actualiser jusqu'à voir l'archive apparaître dans le panneau. 
3.    Exécutez la céllule de code suivante pour déziper l'archive.

In [ ]:
!unzip Archive_Lab7.zip

### Avant propos : un bref rappel du Filtrage Collaboratif

Le problème de la recommandation vise à prédire des scores de pertinence personnalisés à l’utilisateur pour un item, objet non vu. Différentes approches existent : filtrage collaboratif, basées sur le contenu, basées sur la connaissances, ...  (voir le cours 7)

Dans ce Lab on s'intéressera au Filtrage collaboratif. L'idée est de prendre en compte les relations de similarité entre les utilisateurs et les éléments. En d'autres termes, la similarité entre éléments est déterminée en prenant en
compte les notes des utilisateurs ayant jugés ces éléments.

<center><img src='https://drive.google.com/uc?export=view&id=1FjcxWD9IarWw3JAxhvTmagECHoZU-hjp' width=350></center>



Deux types de filtrage collaboratif :

- **Approche par voisinnage (memory-based)** : cette approche calcule la prédiction d'un item $i$ pour un utilisateur $x$ en se basant sur les plus proches voisins de $x$ (i.e. ceux dont l'ensemble de notations est similaire à l'ensemble des notation de $x$). 

- **Approche par modèles (model-based)** : cette approche cherche à predire le score d'un item $i$ pour un utilisateur $x$, sur la base des items qui sont similaires à $i$. Donc, l'estimation de la note pour l’élément se fait à partir des notes des éléments de $x$ similaires à $i$. 

## PARTIE 1 : Exercice

<div class="alert alert-block alert-danger">
<b>Note : </b> un exercice sur un fichier pdf séparé est disponible sur EDUNAO et Teams. Il vous permettra de réviser plus simplement les concepts du cours.
</div>

## PARTIE 2 : Applicaton au corpus MovieLens

Dans cette partie nous travaillerons sur la base du corpus [MOVIELENS dataset](https://grouplens.org/datasets/movielens/) qui contient les jugements de films d'un ensemble d'utilisateurs ainsi que des informations sur les utilisateurs et sur les films. L'objectif de cette partie est de mettre en place un système de recommandation de films à partir de ce corpus. 
Il s'agira par exemple de pouvoir répondre à la question suivante : *quels sont les films pouvant plaire à une personne étant donné la connaissance sur cette personne (e.g. les autres films qu'elle a aimé ou les utilisateurs ressemblant à cette personne) ?* 

Pour ce travail, il est conseillé de travailler avec le [MovieLens 100K Dataset](https://grouplens.org/datasets/movielens/100k/) et de tester ensuite votre approche sur les datasets de  taille plus conséquente :
 + MovieLens 1M Dataset (1M de notes sur 4000 films par 6000 utilisateurs), disponible [ici](https://grouplens.org/datasets/movielens/1m/)
 + MovieLens 10M Dataset (10M de notes sur 10 000 films par 72000 utilisateurs, disponible [ici](https://grouplens.org/datasets/movielens/10m/)
 
Pour répondre aux différentes questions nous allons faire appel à la bibliothque Pandas, numpy et matplotlib, que nous vous proposons d'installer en executant les cellules suivantes. 

In [ ]:
!pip install pandas

In [ ]:
!pip install numpy

In [ ]:
!pip install matplotlib

### Etude du corpus

Une des premières choses à faire est de prendre connaissance de ce corpus et de calculer quelques informations statistiques sur ce corpus. A l'aide des bibliothèques [pandas](https://pandas.pydata.org/), et [numpy](https://www.numpy.org),  répondez aux questions qui vont suivre concernant le corpus. Mais avant cela, nous allons introduire quelques éléments descriptifs utiles sur le corpus.

Le corpus MOVIELENS est disponible [ici](https://grouplens.org/datasets/movielens/100k/) et mis à votre disposition dans le dossier Data.  Plus précisemment, ce corpus est constitué des fichiers suivants :

+ Le fichier [u.data](./Data/ml-100k/u.data) : regroupe l'ensemble des données : 100000 notations par 943 utilisateurs sur 1682 items. Chaque utilisateur  a noté au moins 20 films.  Cette table contient:  

               user id | item id | rating | timestamp.           
          
  les utilisateurs et les items sont numérotés à partir de  1 et les données sont aléatoirement ordonées.  
  
              
+ Le fichier [u.user](./Data/ml-100k/u.user) : information démographique sur les utiliisateurs. Il contient: 
                         
              user id | age | gender | occupation | zip code
                         
                            
+ Le fichier [u.item](./Data/ml-100k/u.item) : contient les informations sur les items (films) : 
              
              movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western |
              


**Le code suivant permet d'extraire le contenu de ces fichiers.** 


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

#column headers for the dataset

data_cols = ['user_id','item_id','rating','timestamp']

#items' table

item_cols = ['movie id','movie title','release date',
'video release date','IMDb URL','unknown','Action',
'Adventure','Animation','Childrens','Comedy','Crime',
'Documentary','Drama','Fantasy','Film-Noir','Horror',
'Musical','Mystery','Romance ','Sci-Fi','Thriller',
'War' ,'Western']


#users' table

user_cols = ['user id','age','gender','occupation',
'zip code']

#importing the data files onto dataframes

# Note: pour les presonnes sur Colab attention a mettre ./Archive_Lab7/Data/ au lieu de ./Data

users = pd.read_csv('./Data/ml-100k/u.user', sep='|',names=user_cols, encoding='latin-1')
item = pd.read_csv('./Data/ml-100k/u.item', sep='|',names=item_cols, encoding='latin-1')
data = pd.read_csv('./Data/ml-100k/u.data', sep='\t',names=data_cols, encoding='latin-1')




In [ ]:
#printing the head of these dataframes

print("User information")
print(users.head()) # it will print only the head. To have the whole table remove .head()


In [ ]:
print("Item information")
print(item.head())


In [ ]:
print("Data information")
print(data.head())

**1- Quel est le nombre de films et d'utilisateurs du corpus ?**



In [ ]:
# TO COMPLETE

nb_users = # to complete
nb_items = #to complete

print("Number of users : ", nb_users)
print("Number of movies : ", nb_items)


**Résultat attendu**


<center><img src='https://drive.google.com/uc?export=view&id=1Uan38PEKTHBukWis-x5-uhBFs_qNupS6' width=300></center>


**2- Contruire la matrice d'utilité**

In [ ]:
#Indication: pivot_table
movie_matrix= #to complete
movi_matrix.head()

**Résultat attendu (here only the head)**


<center><img src='https://drive.google.com/uc?export=view&id=1pHI972WSeIqM_Ua8vlT5VzARHTGarbQy' width=400></center>


**3- Quelle est la note moyenne de chaque film ?** 

Indication: on peut utiliser la fonction `groupby()` de Pandas. Quelques exemples [ici](https://www.tutorialspoint.com/python_pandas/python_pandas_groupby.htm). Sinon ne pas oublier de se reférer à la documentation officielle de Pandas [ici](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html?highlight=groupby#pandas.DataFrame.groupby). 

In [ ]:
#Indication: mettre le résultat dans un DataFrame

ratings = #To compelete
ratings.head()

**Résultat attendu(here only the head)**


<center><img src='https://drive.google.com/uc?export=view&id=1KcS0EhKfWyMJOWn-FYTDDJpu8N1AIROt' width=100></center>

**4- Visualisation de la distribution des notes**

<div class="alert alert-block alert-danger">
<b>Warning: </b> if you did not install yet matplotlip, excute in a separate cell `!pip install matplotlib` 
</div>


Remarque : `%matplotlib`configure la bibliothèque que vous allez utiliser pour dessiner le graphique. Elle effectue donc un certain nombre de traitements pour préparer l'affichage du graphique. Elle est souvent utilisée avec l'argument `inline`, qui indique que l'on va utiliser la bibliothèque intégrée à Notebook. 

Pour la fonction `hist()` des exemples [ici](https://www.science-emergence.com/Articles/Simple-histogramme-avec-matplotlib/)



In [ ]:
# Attention ces instructions ne peuvent être executées sans la réalisation de la question 3. 

import matplotlib.pyplot as plt
%matplotlib inline

ratings['rating'].hist(bins=50)

**Résultat attendu**

<center><img src='https://drive.google.com/uc?export=view&id=1w6fsr9ExDO8fcJBvYylAA6Xc3zD2ySsp' width=300></center>


 **5- Quel est le nombre de notes pour chaque film ?**

In [ ]:
ratings['number_of_ratings'] = # To complete
ratings.head()

**Résultat attendu (head only)**


<center><img src='https://drive.google.com/uc?export=view&id=1OciH_atHuyuWfNInKMN4tZpxnAwwByPR' width=300></center>



**6- Visualiser le nombre de notes pour chaque film (comme nous l'avons fait à la question 4)**

In [ ]:
# To complete

## Système de recommandation

### Préparation de l'évaluation

Une première phase dans le problème de prédiction de la recommandation est de préparer l'étape d'évaluation de votre système de recommandationn en séparant l'ensemble de données en  un ensemble de test et en un ensemble d'apprentissage. 

<center><img src='https://drive.google.com/uc?export=view&id=1i6nY-QOBnZAyWr5dlQraDHHToZLbgqU7' width=300></center>


- L'ensemble d'apprentissage sera utilisé pour construire la matrice d'utilité, calculer le score de similarité et faire la prédiction. 
- L'ensemble de test sera utilisé dans la phase d'évaluation.


<div class="alert alert-block alert-danger">
<b>Warning: </b> Avant de commencer executer la cellule suivante: 
</div> 

In [ ]:
!pip install sklearn

**Séparons nos données `data` en deux sous ensembles : un ensemble d'apprentissage et un ensemble de test. On prendra par exemple une taille d'ensemble de test de $0.25$**.

Indication : on fera appel à la fonction `model_selection()` dont la documentation se trouve [ici](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [ ]:
from sklearn import model_selection 

train_data, test_data = # TO COMPLETE


In [ ]:
print("Taindata\n")
print(train_data)

In [ ]:
print("Testdata\n")
print(test_data)

### Approche par voisinage (user-item, Memory-based)

Il s'agit d'implémenter la méthode de filtrage collaboratif par voisinage **user-item** en suivant les étapes ci-dessous.


### Etape 1 : 

**a- Construire la matrice user-item à partir de l'ensemble des données. (Cela vous permettra de visualiser le contenu des données qui sont considérées pour l'apprentissage.)** 

Attention : on utilisera la matrice `train_data`.

In [ ]:
#indication: utiliser pivot_table

movie_matrix = # TO COMPLETE


**b- Constuire la matrice d'utilité contenant que les notes ou les évaluations des différents items. On appelera cette matrice `train_data_matrix`. C'est cette matrice qui sera utilisée par la suite pour faire nos calculs de similarités et de prédictions.**


Attention : on utilisera la matrice `train_data`.

In [ ]:
import numpy as np

train_data_matrix= #To complete


**Expected result**

<img src="./Figures/result_b.png" width="150" height="150" />

**c- Construire la matrice de test contenant que les évaluation (ou les notations) des différents items. On appelera cette matrice `test_data_matrix`. C'est cette matrice qui sera utilisée par la suite dans la phase d'évaluation**. 

Attention : on utilisera la matrice `test_data`.

In [ ]:
test_data_matrix =# TO COMPLETE


**Expected result**

<img src="./Figures/result_c.png" width="150" height="150" />

### Etape 2 : Construire et Calculer la matrice de similarité entre utilisateurs. 

On utilisera dans un premier temps la mesure **cosinus** : 

$$sim(x,y) = cos (r_x,r_y) = \frac{r_x.r_y}{||r_x||.||r_y||}$$


(on pourra utiliser pour cela la méthode **pairwise\_distances** de scikit-learn documentée [ici]( https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html)).

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

user_similarity= # TO COMPLETE


<div class="alert alert-block alert-danger">
<b>Note : </b> vous pouvez utiliser une autre mesure de similarité comme celle de Pearons (voir plus bas). Notre conseil est de terminer en premier le travail avec la mesure cosinus. Une fois la fonction de prédiction mise en place et que celle-ci fonctionne vous pouvez revenir pour écrire la
    fonction de similarité avec Pearson et la tester ensuite avec la fonction de prédiction. Donc on peut passer à l'étape 3. 
</div>

**La mesure de corrélation de Pearson** 

étant donnés deux utilisateurs $x$ et $y$ :

$$ sim(x,y) = \frac{\sum_{s \in S_{xy}} (r_{xs} - \overline{r_x})(r_{ys} - \overline{r_y}) }{ \sqrt{\sum_{s \in S_{xy}} (r_{xs} - \overline{r_x})^2}   \sqrt{\sum_{s \in S_{xy}} (r_{ys} - \overline{r_y})^2}  }$$

avec $\overline{r_y}$ et  $\overline{r_x}$ la notation moyenne de $x$ et $y$.


**Remarque**: `pearsonr()` da la bibliothèque `stats` retourne un tuple composé du coefficient de corrélation et de la p-valeur:

Le coefficient de corrélation peut varier de -1 à +1. L'hypothèse nulle est que les deux variables ne sont pas corrélées. La valeur de $p$ est un nombre compris entre zéro et celui qui représente la probabilité que vos données se serait produites si l'hypothèse nulle était vraie.
Pour plus de détails, voir http://www.eecs.qmul.ac.uk/~norman/blog_articles/p_values.pdf

In [ ]:
#Exemple
from scipy import stats
import numpy as np

a = np.array([0, 0, 0, 1, 1, 1, 1])
b = np.arange(7)
stats.pearsonr(a, b)[0]

0.8660254037844386

### Etape 3 :  Construction de la matrice de prédictions en utilisant la formule suivante.

La note de l'utilisateur $x$ sur l'élément $i$ est calculée comme suit :

$$ r_{xi} = \overline{r_x} + \frac{\sum_{y \in N} sim(x,y) (r_{yi} -\overline{r_y})}{\sum_{y \in N} | sim(x,y)|}$$


Avec: $\overline{r}_x$ la note moyenne de l'utilisateur $x$,  $\overline{r}_y$ la note moyenne de l'utilisateur $y$ et $r_{yi}$ la note de l'utilisateur $y$ qui a jugé $i$.



**Ecrire la fonction `predict(ratings, similarity, type='user')` qui prendra en entrée les évaluations, la matrice de similarités et un troisième argument qui est `type='user'`. Il est intéressant de le considérer pour pourvoir réutiliser la fonction dans l'approche par modèles en mettant `type='item'`.** 

**Encore une fois vous pouvez écrire simplement la fonction `predict(ratings, similarity)` qui retournera à la fin la matrice de prédictions des différents items pour les différents utilisateurs.**

In [ ]:
def predict(ratings, similarity, type='user'):
    
    # TO COMPLETE
    return pred



In [ ]:
#Test

user_prediction = predict(train_data_matrix, user_similarity, type='user')

### Etape 4 :  Evaluer votre prédiction avec la mesure RMSE (Root Mean Squared Error).


$$\sqrt{\frac{1}{n} \sum_{xi} (r_{xi} - r_{xi}^{*})^{2}}$$
où, $r_{xi}^{*}$ correspond à la valeur prédite pour $r_{xi}$.

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error

def rmse(prediction, ground_truth):
    #To compelte



In [ ]:
#Test

print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))

## Approche par modèles : **item-item**


<div class="alert alert-block alert-danger">
<b>Conseil : </b>  à faire une fois la partie memory-based réalisée. 
</div>

#### **Faire le même travail avec l'approche item-item. Il est possible  de réutiliser les fonctions implémentées précédemment.**

## PARTIE 3 (Optionnelle): Pour aller plus loin... 

Pour aller plus loin et pour les curieux vous pouvez regarder ce [cours/Lab](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/recommendation-systems/recommendation-systems.ipynb?utm_source=ss-recommendation-systems&utm_campaign=colab-external&utm_medium=referral&utm_content=recommendation-systems#scrollTo=9EjQt_o9Xf_L) qui aborde la recommandation avec la technique de factorisation de matrices (matrix factorization model -- non abodée dans le cours) et donc l'utilisation de Tensorflow. 
